Najprej bomo prevedli celoštevilski linearni program iz navodil v funkcijo `alpha_od(G)`

In [1]:
def alpha_od(G):
    """
    Compute the size of the maximum odd independent set in the graph G.
    """
    from sage.numerical.mip import MixedIntegerLinearProgram
    
    n = G.order()
    V = G.vertices()
    
    # Create the mixed integer linear program
    mip = MixedIntegerLinearProgram(maximization=True)
    
    # Define variables
    x = mip.new_variable(binary=True)
    y = mip.new_variable(binary=True)
    z = mip.new_variable(integer=True)
    
    # Objective function
    mip.set_objective(mip.sum(x[v] for v in V))
    
    # Constraints
    for u, v in G.edges(labels=False):
        mip.add_constraint(x[u] + x[v] <= 1)
        
    for u in V:
        sum_neighbors = sum(x[v] for v in G.neighbors(u))
        mip.add_constraint(sum_neighbors <= n * y[u])
        mip.add_constraint(y[u] + sum_neighbors == 2*z[u])
    
    # Solve the MIP
    return mip.solve()

Na nekaj grafih preverimo delovanje (samo test)

In [8]:
from sage.all import Graph

P4 = Graph([(0,1),(1,2),(2,3)])
C4 = Graph([(0,1),(1,2),(2,3),(3,0)])
C5 = Graph([(0,1),(1,2),(2,3),(3,4),(4,0)])

for name, G in [("P4", P4), ("C4", C4), ("C5", C5)]:
    val = alpha_od(G)
    print(name, "alpha_od =", val)

P4 alpha_od = 2.0
C4 alpha_od = 1.0
C5 alpha_od = 1.0
